## Capstone Project : Week 3.1

### Importing the libraries required

In [25]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Web scrapping to obtain the table

In [26]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = BeautifulSoup(html, 'html.parser')
table = content.find('table')
table.prettify()

'<table class="wikitable sortable">\n <tbody>\n  <tr>\n   <th>\n    Postal Code\n   </th>\n   <th>\n    Borough\n   </th>\n   <th>\n    Neighborhood\n   </th>\n  </tr>\n  <tr>\n   <td>\n    M1A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M2A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M3A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Parkwoods\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M4A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Victoria Village\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M5A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>\n    Regent Park, Harbourfront\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M6A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Lawrence Manor, Lawrence Heights\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M7A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>\n    Queen\'s Park, Ontario Provinci

### Extracting every row from the table and adding it to the new data frame

In [28]:
data = []
for tr in table.find_all('tr')[1:]:
    row_data = tr.find_all('td')
    data.append([cell.text.rstrip("\n") for cell in row_data])
dataset = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Dropping the rows which have "not assigned" values for Borough

In [30]:
dataset = dataset[dataset['Borough'] != 'Not assigned']
dataset.reset_index(drop = True, inplace = True)
dataset.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Assign Neighbourhood to same as Borough if it is "Not assigned".

In [31]:
dataset['Neighborhood'] = dataset['Neighborhood'].str.split('\n', expand = True)[0]
dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Neighborhood'] = dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Borough']

### Grouping with respect to PostalCode and Borough

In [32]:
dataset = dataset.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [33]:
dataset.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### The shape of the final dataset
##### I will save the new dataset as a csv file for future purposes

In [35]:
dataset.to_csv('dataset.csv', index = False)
print("Dataset shape: {}".format(dataset.shape))

Dataset shape: (103, 3)
